In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.models import load_model
from tensorflow.keras.utils  import to_categorical
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input

from sklearn.preprocessing import MinMaxScaler




In [3]:
df = pd.read_csv("/kaggle/input/mobile-health/mhealth_raw_data.csv")
df.head()

In [4]:
df.info()

In [5]:
df['Activity'].value_counts()

activity_labels = 
    0: "None",
    1: "Standing still (1 min)",
    2: "Sitting and relaxing (1 min)",
    3: "Lying down (1 min)",
    4: "Walking (1 min)",
    5: "Climbing stairs (1 min)",
    6: "Waist bends forward (20x)",
    7: "Frontal elevation of arms (20x)",
    8: "Knees bending (crouching) (20x)",
    9: "Cycling (1 min)",
    10: "Jogging (1 min)",
    11: "Running (1 min)",
    12: "Jump front & back (20x)"

In [6]:
df = df[(df.Activity != 0) & (df.Activity != 2) & (df.Activity != 3)]
df.Activity.value_counts()

In [7]:
df.subject.value_counts()

In [8]:
# creating equal group sizes 
df = df.groupby(['subject', 'Activity']).apply(lambda grp: grp.sample(n=1000))

In [9]:
df.reset_index(drop=True, inplace=True)

In [10]:
df.head()

In [11]:
df.Activity.value_counts()

In [12]:
df = df.drop(columns=['subject'])

In [13]:
train_size = int(len(df)*0.7)
train, test = df.iloc[:train_size],df.iloc[train_size:]

In [14]:
X_train = train.drop('Activity', axis = 1)
y_train = train.loc[:,['Activity']]

# Split test data to X and y
X_test = test.drop('Activity', axis = 1)
y_test = test.loc[:,['Activity']]

In [15]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils  import to_categorical

# Different scaler for input and output
scaler_x = MinMaxScaler(feature_range = (0,1))

# Fit the scaler using available training data
input_scaler = scaler_x.fit(X_train)

# Apply the scaler to training data
train_x_norm = input_scaler.transform(X_train)

# Apply the scaler to test data
test_x_norm = input_scaler.transform(X_test)

# one-hot encode y
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [16]:
def threeD_dataset (X, y, time_steps = 1):
    Xs, ys = [], []
    
    for i in range(len(X)-time_steps):
        v = X[i:i+time_steps, :]
        Xs.append(v)
        ys.append(y[i+time_steps])
        
    return np.array(Xs), np.array(ys)


TIME_STEPS = 50

X_test, y_test = threeD_dataset(test_x_norm, y_test, TIME_STEPS)
X_train, y_train = threeD_dataset(train_x_norm, y_train, TIME_STEPS)

print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape) 
print('y_test.shape: ', y_test.shape)

In [17]:
# fit and evaluate a model
def fit_evaluate_model(X_train, y_train, X_test, y_test, n_filters):
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
    # create model
    model = Sequential()
    model.add(Conv1D(filters=n_filters, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=n_filters, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    plot_model(model, show_shapes=True, to_file='sequential.png')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
    return accuracy
 
# summarize performance (accuracy, mean, sd)
def performance_summary(scores, params):
    print(scores, params)
    for score in range(len(scores)):
        m, s = mean(scores[score]), std(scores[score])
        print('Filter=%d: %.3f%% (+/-%.3f)' % (params[score], m, s))
    # boxplot of scores
    plt.boxplot(scores, labels=params)
    plt.savefig('cnn_filters.png')
 
# run the trials
def run_trials(params, repeats=5):
    # test each parameter
    all_scores = list()
    for param in params:
        scores = list()
        for repeat in range(repeats):
            score = fit_evaluate_model(X_train, y_train, X_test, y_test, param)
            score = score * 100.0
            print('>Filter=%d #%d: %.3f' % (param, repeat+1, score))
            scores.append(score)
        all_scores.append(scores)
    performance_summary(all_scores, params)
 
# run 
n_filters = [8, 16, 32, 64, 128, 256]
run_trials(n_filters)

In [18]:
# fit and evaluate a model
def fit_evaluate_model(X_train, y_train, X_test, y_test, n_kernel):
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
    # create model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=n_kernel, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=n_kernel, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
    return accuracy
 
# summarize performance (accuracy, mean, sd)
def performance_summary(scores, params):
    print(scores, params)
    for score in range(len(scores)):
        m, s = mean(scores[score]), std(scores[score])
        print('Kernel=%d: %.3f%% (+/-%.3f)' % (params[score], m, s))
    # boxplot of scores
    plt.boxplot(scores, labels=params)
    plt.savefig('cnn_filters.png')
 
# run the trials
def run_trials(params, repeats=5):
    # test each parameter
    all_scores = list()
    for param in params:
        scores = list()
        for repeat in range(repeats):
            score = fit_evaluate_model(X_train, y_train, X_test, y_test, param)
            score = score * 100.0
            print('>Kernel=%d #%d: %.3f' % (param, repeat+1, score))
            scores.append(score)
        all_scores.append(scores)
    performance_summary(all_scores, params)
 
# run
n_kernels = [2, 3, 5, 7, 11]
run_trials(n_kernels)

In [19]:
# fit and evaluate a model
def fit_evaluate_model(X_train, y_train, X_test, y_test, n_dropout):
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
    # create model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(n_dropout))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
    return accuracy
 
# summarize performance (accuracy, mean, sd)
def performance_summary(scores, params):
    print(scores, params)
    for score in range(len(scores)):
        m, s = mean(scores[score]), std(scores[score])
        print('Dropout=%.1f: %.3f%% (+/-%.3f)' % (params[score], m, s))
    # boxplot of scores
    plt.boxplot(scores, labels=params)
    plt.savefig('cnn_filters.png')
 
# run the trials
def run_trials(params, repeats=5):
    # test each parameter
    all_scores = list()
    for param in params:
        scores = list()
        for repeat in range(repeats):
            score = fit_evaluate_model(X_train, y_train, X_test, y_test, param)
            score = score * 100.0
            print('>Dropout=%.1f #%d: %.3f' % (param, repeat+1, score))
            scores.append(score)
        all_scores.append(scores)
    performance_summary(all_scores, params)
 
# run 
n_dropout = [0.1, 0.3, 0.5, 0.7, 0.9]
run_trials(n_dropout)

In [20]:
from keras.layers.merge import concatenate
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
# fit and evaluate a model
def fit_evaluate_model(X_train, y_train, X_test, y_test):
    # build the model
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
    # input head 1
    inputs1 = Input(shape=(n_timesteps,n_features))
    conv1 = Conv1D(filters=256, kernel_size=2, activation='relu')(inputs1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    # input head 2
    inputs2 = Input(shape=(n_timesteps,n_features))
    conv2 = Conv1D(filters=128, kernel_size=3, activation='relu')(inputs2)
    drop2 = Dropout(0.3)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    # input head 3
    inputs3 = Input(shape=(n_timesteps,n_features))
    conv3 = Conv1D(filters=64, kernel_size=5, activation='relu')(inputs3)
    drop3 = Dropout(0.1)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    # concatenate
    merged = concatenate([flat1, flat2, flat3])
    # interpretation
    dense1 = Dense(100, activation='relu')(merged)
    outputs = Dense(n_outputs, activation='softmax')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    # save a plot of the model
    plot_model(model, show_shapes=True, to_file='multichannel.png')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit([X_train,X_train,X_train], y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate([X_test,X_test,X_test], y_test, batch_size=batch_size, verbose=verbose)
    return accuracy
 
# summarize performance (accuracy, mean, sd)
def performance_summary(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 
# run the trials
def run_trials(repeats=5):
    # repeat experiment
    scores = list()
    for repeat in range(repeats):
        score = fit_evaluate_model(X_train, y_train, X_test, y_test)
        score = score * 100.0
        print('>#%d: %.3f' % (repeat+1, score))
        scores.append(score)
    # performance summary 
    performance_summary(scores)
 
# run 
run_trials()